In [1]:
import pyspark

In [14]:
## Pyspark
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType

## Numerical frameworks
import numpy as np
import pandas as pd

## Matplotlib
import matplotlib.pyplot as plt

## Bokeh
from bokeh.io import show, output_file, output_notebook
from bokeh.palettes import Spectral6
from bokeh.plotting import figure
from bokeh.resources import CDN
from bokeh.embed import file_html
from bokeh.transform import factor_cmap
from bokeh.io import show
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LinearColorMapper,
    BasicTicker,
    PrintfTickFormatter,
    ColorBar
)

from bokeh.layouts import column, row, WidgetBox
from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

In [3]:
sc = SparkContext.getOrCreate()
sqlContext = SQLContext(sc)

In [4]:
df = sqlContext.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true')\
    .load('data.csv')

In [5]:
df.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- Community Area: integer (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: integer (nullable = true)
 |-- Y Coordinate: integer (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longitude: double (nullable = true)
 |-- Location: string (nullable = true)



In [6]:
df.groupBy("Primary Type") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|               THEFT|1371774|
|             BATTERY|1196547|
|     CRIMINAL DAMAGE| 751428|
|           NARCOTICS| 701699|
|       OTHER OFFENSE| 406530|
|             ASSAULT| 403897|
|            BURGLARY| 379662|
| MOTOR VEHICLE THEFT| 307555|
|             ROBBERY| 249023|
|  DECEPTIVE PRACTICE| 248402|
|   CRIMINAL TRESPASS| 188642|
|        PROSTITUTION|  67754|
|   WEAPONS VIOLATION|  66762|
|PUBLIC PEACE VIOL...|  46812|
|OFFENSE INVOLVING...|  43603|
| CRIM SEXUAL ASSAULT|  25816|
|         SEX OFFENSE|  24164|
|            GAMBLING|  14234|
|INTERFERENCE WITH...|  14209|
|LIQUOR LAW VIOLATION|  13856|
+--------------------+-------+
only showing top 20 rows



### Number of crime over the years

In [7]:
year_list = df.groupBy("Year") \
    .count() \
    .orderBy(col("count").desc()).toPandas()

In [60]:
p = figure(title="Chicago crimes history", x_axis_location="below", plot_width=700, plot_height=300,
           toolbar_location='right', x_axis_label="Years", y_axis_label="Number of crimes (thousand)")

p.vbar( x=year_list['Year'], top = year_list['count']/1e3, width=0.8, fill_alpha=0.6,
       fill_color='red', line_color='black')

output_notebook()
show(p)

Loading BokehJS ...

In [35]:
# bokeh basics
from bokeh.plotting import figure
from bokeh.io import show, output_notebook

# Create a blank figure with labels
p = figure(plot_width = 600, plot_height = 600, 
           title = 'Example Glyphs',
           x_axis_label = 'X', y_axis_label = 'Y')

# Example data
squares_x = [1, 3, 4, 5, 8]
squares_y = [8, 7, 3, 1, 10]
circles_x = [9, 12, 4, 3, 15]
circles_y = [8, 4, 11, 6, 10]

# Add squares glyph
p.square(squares_x, squares_y, size = 12, color = 'navy', alpha = 0.6)
# Add circle glyph
p.circle(circles_x, circles_y, size = 12, color = 'red')

# Set to output the plot in the notebook
output_notebook()
# Show the plot
show(p)

Loading BokehJS ...

In [74]:
split_col = pyspark.sql.functions.split(df['Date'], ' ')
df_yr_split = df.withColumn('date_only', split_col.getItem(0)).withColumn('time_only', split_col.getItem(1))

In [75]:
split_col = pyspark.sql.functions.split(df_yr_split['date_only'], '/')
df_date = df_yr_split.withColumn('Month', split_col.getItem(0).cast('int')).withColumn('Day', split_col.getItem(1).cast('int'))

In [15]:
yr_month.head()

NameError: name 'yr_month' is not defined

In [77]:
df_date_new = df_date.select(col("Primary Type").alias("crime_type"), col("Month").alias("Month"), col("Year").alias("Year"))

In [80]:
df_date_new.printSchema()

root
 |-- crime_type: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [81]:
yr_month_theft = df_date_new.filter(df_date_new.crime_type == 'THEFT')\
                        .groupby('Year', 'Month')\
                        .count()\
                        .orderBy(col('Year')\
                        .desc()).toPandas()

In [56]:
def make_plot(yr_month):

    months = ['January', 'February', 'March', 'April', 'May', 'June', 'July', \
              'August', 'September', 'October', 'November', 'December']
    yr_month['Month'] = yr_month['Month'].map(lambda x: months[int(x)-1])

    years = np.unique(yr_month['Year'])
    number_of_months = 12
    yr_month_matrix = np.zeros((len(years), number_of_months))

    for i, year in enumerate(years):
        no_yr = yr_month[yr_month['Year'] == year]

        for j in range(len(no_yr)):
            yr_month_matrix[i][j] = no_yr.iloc[j]['count']

    yr_month_matrix = pd.DataFrame(yr_month_matrix, columns = months)

    yr_month_matrix['Year'] = years
    data = yr_month_matrix

    data['Year'] = data['Year'].astype(str)
    data = data.set_index('Year')
    data.columns.name = 'Month'

    years = list(data.index)
    months = list(data.columns)

    df_yr = pd.DataFrame(data.stack(), columns=['rate']).reset_index()

    # this is the colormap from the original NYTimes plot
    colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]

    mapper = LinearColorMapper(palette=colors, low=df_yr.rate.min(), high=df_yr.rate.max())
    source = ColumnDataSource(df_yr)

    p = figure(title="Chicago crimes ({0} - {1})".format(years[0], years[-1]),
               x_range= years, y_range = list(reversed(months)),
               x_axis_location="above", plot_width=900, plot_height=400,
               toolbar_location='below')

    p.rect(x="Year", y="Month", width=1, height=1,
           source=source,
           fill_color={'field': 'rate', 'transform': mapper},
           line_color='black')

    color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="6pt",
                         ticker=BasicTicker(desired_num_ticks=len(colors)),
                         label_standoff = 6, border_line_color= None, location=(0, 0))

    p.add_layout(color_bar, 'right')

    p.grid.grid_line_color = None
    p.axis.axis_line_color = None
    p.axis.major_tick_line_color = None
    p.axis.major_label_text_font_size = "12pt"
    p.axis.major_label_standoff = 0
    p.xaxis.major_label_orientation = 1.55

    # Add a hover tool referring to the formatted columns
    hover = HoverTool(tooltips = [('Date', '@Month @Year'),('No of crimes', '@rate')])
    p.add_tools(hover)
    return p

In [57]:
show(make_plot(yr_month))

In [82]:
show(make_plot(yr_month_theft))

In [40]:
df_yr = pd.DataFrame(data.stack(), columns=['rate']).reset_index()

In [36]:
yr_month_matrix['Year'] = years
data = yr_month_matrix

## Interactive individual crime vizualization

In [29]:
from bokeh.models.widgets import CheckboxGroup
from bokeh.models.widgets import RadioButtonGroup

In [96]:
individual_crimes = df.groupBy("Primary Type") \
                      .count() \
                      .orderBy(col("count").desc()) \
                      .toPandas()

In [123]:
crimes_chk = CheckboxGroup(labels = list(individual_crimes['Primary Type'][0:10]), 
                                  active = [0, 1]) 

In [125]:
crimes_rdbtn = RadioButtonGroup(labels = list(individual_crimes['Primary Type'][0:10])) 

In [134]:
def modify_doc(doc):
    
    def make_dataset(crime_type):
        ind_crime = df_date.filter(df_date["Primary Type"] == crime_type)\
        .groupby('Year', 'Month') \
        .count()\
        .orderBy(col('Year')\
        .desc())\
        .toPandas()

        return ind_crime_count

    def update(attr, old, new):
        new_src = make_dataset(crime_type)
        src.data.update(new_src.data)
        

    crimes = CheckboxGroup(labels=available_carriers, active = [0, 1])
    crimes.on_change('active', update)

    controls = WidgetBox(crimes)

    selected_crime = crimes.active

    src = make_dataset(selected_crime)

    p = make_plot(src)

    layout = row(controls, p)
    doc.add_root(layout)

In [41]:
for w in [mpg, cyl, hp, yr, origin]:
    w.on_change('value', update_data)

# Set up layouts and add to document
inputs_1 = widgetbox(mpg, cyl, hp)
inputs_2 = widgetbox(yr, origin)
inputs_row = row(inputs_1, inputs_2, width=800)
layout = column(inputs_row, p, width=800)

curdoc().add_root(layout)
curdoc().title = "AutoMPG"

In [33]:
autompg

,mpg,cyl,displ,hp,weight,accel,yr,origin,name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
5,15.0,8,429.0,198,4341,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220,4354,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215,4312,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225,4425,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190,3850,8.5,70,1,amc ambassador dpl


In [32]:
import numpy as np

from bokeh.io import curdoc
from bokeh.layouts import row, column, widgetbox, gridplot
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, TextInput, MultiSelect
from bokeh.plotting import figure
from bokeh.sampledata.autompg import autompg

def make_plot(df):

	#Set up data
	source = ColumnDataSource(df)

	#Set up plot
	options = dict(plot_width=250, plot_height=250,
	               tools="pan,wheel_zoom,box_zoom,box_select,lasso_select,reset")
	p1 = figure(title="MPG by Year", **options)
	p1.circle("yr", "mpg", color="blue", source=source)
	p2 = figure(title="HP vs. Displacement", **options)
	p2.circle("hp", "displ", color="green", source=source)
	p3 = figure(title="MPG vs. Displacement", **options)
	p3.circle("mpg", "displ", size="cyl", line_color="red", fill_color=None, source=source)
	p = gridplot([[p1, p2, p3]], toolbar_location="right")

	return p

p = make_plot(autompg)

# Set up widgets
mpg = Slider(title="mpg", value=9.0, start=9.0, end=50.0, step=1)
cyl = Slider(title="cyl", value=3.0, start=3.0, end=8.0, step=1)
hp = Slider(title="hp", value=40, start=40, end=240, step=20)
yr = Slider(title="yr", value=70, start=70, end=82, step=1)
origin = MultiSelect(title="origin", value=["1", "2", "3"],
                     options=[("1", "Japanese"), ("2", "German"), ("3", "American")])

#Set up callbacks
def update_data(attrname, old, new):

    # Get the current slider values
    m = mpg.value
    c = cyl.value
    h = hp.value
    y = yr.value
    o = origin.value
    o_int = [int(i) for i in o]

    # Generate new data
    data = autompg[(autompg.mpg >= m)
    			   & (autompg.cyl >= c)
    			   & (autompg.hp >= h)
    			   & (autompg.yr >= y)
    			   & (autompg.origin.isin(o_int))].reset_index(drop=True)
    
    layout.children[1] = make_plot(data)
    

for w in [mpg, cyl, hp, yr, origin]:
    w.on_change('value', update_data)

# Set up layouts and add to document
inputs_1 = widgetbox(mpg, cyl, hp)
inputs_2 = widgetbox(yr, origin)
inputs_row = row(inputs_1, inputs_2, width=800)
layout = column(inputs_row, p, width=800)

curdoc().add_root(layout)
curdoc().title = "AutoMPG"